In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
wiki_xml = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_U.S._states_and_territories_by_income_inequality&oldid=949833744").text
soup = BeautifulSoup(wiki_xml)
gini_df = pd.read_html(str(soup.find('table', class_='wikitable')))[0]

gini_df['state'] = gini_df['State or federal district']
gini_df = gini_df[['state', 'Gini Coefficient']].copy()

gini_df.state.replace('District of Columbia', 'District Of Columbia', inplace=True)

In [3]:
wiki_xml = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_U.S._states_and_territories_by_income&oldid=947939623").text
soup = BeautifulSoup(wiki_xml)
income_df = pd.read_html(str(soup.find('table', class_='wikitable')))[0]
income_df = income_df[income_df['2016'] != '—']

income_df['state'] = income_df['State or territory']
income_df['income'] = income_df['2016'].apply(lambda x: x[1:].replace(',', '')).astype(float)
income_df = income_df[['state', 'income']].copy()

income_df.state.replace('Washington, D.C.', 'District Of Columbia', inplace=True)

In [4]:
votes_state = pd.read_csv("../data/raw/2016ElectionResultsByState.csv")
votes_state.state.replace('District of Columbia', 'District Of Columbia', inplace=True)
votes_state = votes_state[['state', 'clintonVotes', 'trumpVotes', 'totalVotes']].copy()
votes_state['swing'] = (
    ((votes_state.clintonVotes / votes_state.trumpVotes) < 1.04)
    &
    ((votes_state.trumpVotes / votes_state.clintonVotes) < 1.04)
)

In [5]:
education_xml = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_U.S._states_and_territories_by_educational_attainment&oldid=961454186").text
soup = BeautifulSoup(education_xml)
education = pd.read_html(str(soup.find('table', class_='wikitable')))[0]
education.columns = ['state'] + list(education.columns[1:])
education.state.replace('District of Columbia', 'District Of Columbia', inplace=True)
education['with_diploma'] = education['% High school graduate or higher'].apply(lambda x: float(x[:-1])/100.)
education = education[['state', 'with_diploma']].copy()

In [6]:
unemp = pd.read_csv("../data/raw/unemployment.csv")
unemp['unemployment2016'] = unemp.unemployment2016 / 100.

In [7]:
(votes_state
    .merge(income_df, how='left')
    .merge(gini_df, how='left')
    .merge(education, how='left')
    .merge(unemp, how='left')
).to_csv("../data/processed/states-extra.csv", index=False)